In [2]:
# Load MVBench dataset

import argparse
import itertools
import json
import os
import random
import time
from functools import partial

import cv2
import imageio
import torchvision.transforms as T
import numpy as np
import torch
from decord import VideoReader, cpu
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms.functional import InterpolationMode
from tqdm import tqdm

# build_transform, dynamic_preprocess
def build_transform(is_train, input_size, pad2square=False, normalize_type='imagenet'):
    if normalize_type == 'imagenet':
        MEAN, STD = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
    elif normalize_type == 'clip':
        MEAN, STD = (0.4814546, 0.4578275, 0.40821073), (0.2686295, 0.2613025, 0.2757711)
    elif normalize_type == 'siglip':
        MEAN, STD = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
    else:
        raise NotImplementedError
    if is_train:  # use data augumentation
        transform = T.Compose([
            T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
            T.RandomChoice([T.Lambda(jpeg_degrade_functions[quality]) for quality in qualities]),
            T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
            T.ToTensor(),
            T.Normalize(mean=MEAN, std=STD)
        ])
    else:
        if pad2square is False:  # now we use this transform function by default
            transform = T.Compose([
                T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
                T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
                T.ToTensor(),
                T.Normalize(mean=MEAN, std=STD)
            ])
        else:
            transform = T.Compose([
                T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
                T.Lambda(lambda img: expand2square(img, tuple(int(x * 255) for x in MEAN))),
                T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
                T.ToTensor(),
                T.Normalize(mean=MEAN, std=STD)
            ])

    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    # print(f'width: {width}, height: {height}, best_ratio: {best_ratio}')
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=6, image_size=448, use_thumbnail=False):
    
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

data_list = {
    'Action Sequence': ('action_sequence.json', 'MVBench/video/star/Charades_v1_480/', 'video', True),
    # has start & end
    'Action Prediction': ('action_prediction.json', 'MVBench/video/star/Charades_v1_480/', 'video', True),
    # has start & end
    'Action Antonym': ('action_antonym.json', 'MVBench/video/ssv2_video/', 'video', False),
    'Fine-grained Action': (
    'fine_grained_action.json', 'MVBench/video/Moments_in_Time_Raw/videos/', 'video', False),
    'Unexpected Action': ('unexpected_action.json', 'MVBench/video/FunQA_test/test/', 'video', False),
    'Object Existence': ('object_existence.json', 'MVBench/video/clevrer/video_validation/', 'video', False),
    'Object Interaction': ('object_interaction.json', 'MVBench/video/star/Charades_v1_480/', 'video', True),
    # has start & end
    'Object Shuffle': ('object_shuffle.json', 'MVBench/video/perception/videos/', 'video', False),
    'Moving Direction': ('moving_direction.json', 'MVBench/video/clevrer/video_validation/', 'video', False),
    'Action Localization': ('action_localization.json', 'MVBench/video/sta/sta_video/', 'video', True),
    # has start & end
    'Scene Transition': ('scene_transition.json', 'MVBench/video/scene_qa/video/', 'video', False),
    'Action Count': ('action_count.json', 'MVBench/video/perception/videos/', 'video', False),
    'Moving Count': ('moving_count.json', 'MVBench/video/clevrer/video_validation/', 'video', False),
    'Moving Attribute': ('moving_attribute.json', 'MVBench/video/clevrer/video_validation/', 'video', False),
    'State Change': ('state_change.json', 'MVBench/video/perception/videos/', 'video', False),
    'Fine-grained Pose': ('fine_grained_pose.json', 'MVBench/video/nturgbd/', 'video', False),
    'Character Order': ('character_order.json', 'MVBench/video/perception/videos/', 'video', False),
    'Egocentric Navigation': ('egocentric_navigation.json', 'MVBench/video/vlnqa/', 'video', False),
    'Episodic Reasoning': ('episodic_reasoning.json', 'MVBench/video/tvqa/frames_fps3_hq/', 'frame', True),
    # has start & end, read frame
    'Counterfactual Inference': (
    'counterfactual_inference.json', 'MVBench/video/clevrer/video_validation/', 'video', False),
}

data_dir = './MVBench/json'

class MVBenchDataset(torch.utils.data.Dataset):

    def __init__(self, data_dir, data_list, prompt, question_prompt, num_segments=16, input_size=224,
                 dynamic_image_size=False, use_thumbnail=False, max_num=6):
        
        self.data_list = []

        self.prompt = prompt
        self.question_prompt = question_prompt
        self.input_size = input_size
        self.num_segments = num_segments
        self.dynamic_image_size = dynamic_image_size
        self.use_thumbnail = use_thumbnail
        self.max_num = max_num
        self.transform = build_transform(is_train=False, input_size=input_size)
        
        for k, v in data_list.items():
            with open(os.path.join(data_dir, v[0]), 'r') as f:
                json_data = json.load(f)
            for data in json_data:
                video_path = os.path.join(v[1], data['video'])
                if os.path.exists(video_path):  # 只添加存在的视频文件对应的数据
                    self.data_list.append({
                        'task_type': k,
                        'prefix': v[1],
                        'data_type': v[2],
                        'bound': v[3],
                        'data': data
                    })

        self.decord_method = {
            'video': self.read_video,
            'gif': self.read_gif,
            'frame': self.read_frame,
        }

    def __len__(self):
        return len(self.data_list)

    def __str__(self):
        len_list = {}
        option_list = {}
        for data in self.data_list:
            if data['task_type'] not in len_list:
                len_list[data['task_type']] = 0
            len_list[data['task_type']] += 1
            if data['task_type'] not in option_list:
                option_list[data['task_type']] = 0
            option_list[data['task_type']] += len(data['data']['candidates'])

        correct = 0
        total = 0
        res = f'There are {len(self.data_list)} videos as follow:\n'
        for k, v in len_list.items():
            correct += len_list[k]
            total += option_list[k]
            res += f'{v} for {k} ({option_list[k]} options => {len_list[k] / option_list[k] * 100:.2f}%)\n'
            correct = correct + 1 / option_list[k]
        res += f'Total random accuracy: {correct / total * 100:.2f}%'
        return res.rstrip()

    def get_index(self, bound, fps, max_frame, first_idx=0):
        if bound:
            start, end = bound[0], bound[1]
        else:
            start, end = -100000, 100000
        start_idx = max(first_idx, round(start * fps))
        end_idx = min(round(end * fps), max_frame)
        seg_size = float(end_idx - start_idx) / self.num_segments
        frame_indices = np.array([
            int(start_idx + (seg_size / 2) + np.round(seg_size * idx))
            for idx in range(self.num_segments)
        ])
        return frame_indices

    def read_video(self, video_path, bound=None):
        vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
        max_frame = len(vr) - 1
        fps = float(vr.get_avg_fps())

        images_group = list()
        frame_indices = self.get_index(bound, fps, max_frame, first_idx=0)
        for frame_index in frame_indices:
            img = Image.fromarray(vr[frame_index].asnumpy())
            images_group.append(img)

        return images_group

    def read_gif(self, video_path, bound=None, fps=25):
        gif = imageio.get_reader(video_path)
        max_frame = len(gif) - 1

        images_group = list()
        frame_indices = self.get_index(bound, fps, max_frame, first_idx=0)
        for index, frame in enumerate(gif):
            if index in frame_indices:
                img = cv2.cvtColor(frame, cv2.COLOR_RGBA2RGB)
                img = Image.fromarray(img)
                images_group.append(img)

        return images_group

    def read_frame(self, video_path, bound=None, fps=3):
        max_frame = len(os.listdir(video_path))
        images_group = list()
        frame_indices = self.get_index(bound, fps, max_frame, first_idx=1)  # frame_idx starts from 1
        for frame_index in frame_indices:
            img = Image.open(os.path.join(video_path, f'{frame_index:05d}.jpg'))
            images_group.append(img)

        return images_group

    def qa_template(self, data):
        question = f"Question: {data['question']}\n"
        question += 'Options:\n'
        answer = data['answer']
        answer_idx = -1
        for idx, c in enumerate(data['candidates']):
            question += f"({chr(ord('A') + idx)}) {c}\n"
            if c == answer:
                answer_idx = idx
        question = question.rstrip()
        answer = f"({chr(ord('A') + answer_idx)}) {answer}"
        return question, answer

    def __getitem__(self, idx):
        data_item = self.data_list[idx]
        video_path = os.path.join(data_item['prefix'], data_item['data']['video'])
        
        # 检查视频文件是否存在
        if not os.path.exists(video_path):
            print(f"Warning: Video file not found at {video_path}, skipping this item.")
            return None  # 或者返回一个特定的占位符数据结构
    
        decord_method = self.decord_method[data_item['data_type']]
        bound = None
        if data_item['bound']:
            bound = (data_item['data']['start'], data_item['data']['end'])
    
        image_list = decord_method(video_path, bound)
        if not image_list:  # 检查返回的列表是否为空
            print(f"Warning: No images were extracted from video at {video_path}, skipping this item.")
            return None  # 或者返回一个特定的占位符数据结构
    
        # 处理图像和问题-答案数据
        special_tokens = '\n'.join(['Frame{}: <image>'.format(i + 1) for i in range(len(image_list))])
        question, answer = self.qa_template(data_item['data'])
        question = special_tokens + '\n' + self.prompt + '\n' + question + self.question_prompt
    
        raw_images = []
        num_patches_list = []
        pixel_values = []
        for image in image_list:
            raw_images.append(image)
            if self.dynamic_image_size:
                patches = dynamic_preprocess(image, image_size=self.input_size,
                                             use_thumbnail=self.use_thumbnail,
                                             max_num=self.max_num)
            else:
                patches = [image]
            num_patches_list.append(len(patches))
            pixel_values.extend([self.transform(patch) for patch in patches])
    
        if not pixel_values:  # 检查是否有有效的像素值数据
            return None  # 返回占位符或None
    
        pixel_values = torch.stack(pixel_values)
    
        return {
            'question': question,
            'pixel_values': pixel_values,
            'answer': answer,
            'num_patches_list': num_patches_list,
            'task_type': data_item['task_type']
        }

In [3]:
prompt = 'Carefully watch the video and pay attention to the cause and sequence of events, the detail and movement of objects, and the action and pose of persons. Based on your observations, select the best option that accurately addresses the question.\n'
question_prompt = '\nOnly give the best option.'

dataset = MVBenchDataset(
    data_dir=data_dir,          
    data_list=data_list,        
    prompt=prompt,               
    question_prompt=question_prompt,
    num_segments=8,              
    input_size=448,              
    dynamic_image_size=True,     
    use_thumbnail=False,          
    max_num=6                     
)

FileNotFoundError: [Errno 2] No such file or directory: 'MVBench/json/action_sequence.json'

In [3]:
print(dataset)

There are 3986 videos as follow:
188 for Action Sequence (752 options => 25.00%)
200 for Action Prediction (800 options => 25.00%)
200 for Action Antonym (600 options => 33.33%)
200 for Fine-grained Action (800 options => 25.00%)
200 for Unexpected Action (800 options => 25.00%)
198 for Object Existence (594 options => 33.33%)
200 for Object Interaction (800 options => 25.00%)
200 for Object Shuffle (600 options => 33.33%)
200 for Moving Direction (800 options => 25.00%)
200 for Action Localization (800 options => 25.00%)
200 for Scene Transition (800 options => 25.00%)
200 for Action Count (600 options => 33.33%)
200 for Moving Count (800 options => 25.00%)
200 for Moving Attribute (601 options => 33.28%)
200 for State Change (600 options => 33.33%)
200 for Fine-grained Pose (800 options => 25.00%)
200 for Character Order (600 options => 33.33%)
200 for Egocentric Navigation (800 options => 25.00%)
200 for Episodic Reasoning (1000 options => 20.00%)
200 for Counterfactual Inference (6

In [4]:
from transformers import AutoModel, AutoTokenizer, AutoProcessor

device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")
model_name = "OpenGVLab/InternVL2-2B"
model = AutoModel.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.bfloat16).eval()

model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)

InternLM2ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


The repository for OpenGVLab/InternVL2-2B contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/OpenGVLab/InternVL2-2B.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


In [5]:
import torch
from torch.utils.data import DataLoader
from functools import partial


def collate_fn(batches, tokenizer):
    
    pixel_values = torch.cat([_['pixel_values'] for _ in batches], dim=0)
    questions = [_['question'] for _ in batches]
    answers = [_['answer'] for _ in batches]
    num_patches_lists = [_['num_patches_list'] for _ in batches]
    task_types = [_['task_type'] for _ in batches]
    
    return pixel_values, questions, answers, num_patches_lists, task_types

dataloader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=1,
        num_workers=1,
        pin_memory=True,
        drop_last=False,
        collate_fn=partial(collate_fn, tokenizer=tokenizer)
    )

first_batch = next(iter(dataloader))

pixel_values = first_batch[0]
question = first_batch[1]
answer = first_batch[2]
num_patches_list = first_batch[3][0]
task_type = first_batch[4]

print(pixel_values.size())
print(question)
print(answer)
print(num_patches_list)
print(task_type)

torch.Size([16, 3, 448, 448])
['Frame1: <image>\nFrame2: <image>\nFrame3: <image>\nFrame4: <image>\nFrame5: <image>\nFrame6: <image>\nFrame7: <image>\nFrame8: <image>\nCarefully watch the video and pay attention to the cause and sequence of events, the detail and movement of objects, and the action and pose of persons. Based on your observations, select the best option that accurately addresses the question.\n\nQuestion: What happened after the person took the food?\nOptions:\n(A) Ate the medicine.\n(B) Tidied up the blanket.\n(C) Put down the cup/glass/bottle.\n(D) Took the box.\nOnly give the best option.']
['(A) Ate the medicine.']
[2, 2, 2, 2, 2, 2, 2, 2]
['Action Sequence']


In [6]:
dataloader

In [7]:
model.dtype

torch.bfloat16

In [8]:
generation_config = dict(max_new_tokens=1024, do_sample=False)
epoch = 1

y_pred = []
y_real = []

progress_bar = tqdm(
        dataloader, total=len(dataloader), desc=f"Epoch {epoch}/1"
    )

for step, batch in enumerate(progress_bar, start=1):
        
    pixel_values = batch[0].to(torch.bfloat16).to(device)
    question = batch[1][0]
    answer = batch[2]
    num_patches_list = batch[3][0]
    task_type = batch[4]

    # print(pixel_values.size())
    # print(question)
    # print(answer)
    # print(num_patches_list)
    # print(task_type)
    
    # with autocast():
        # 执行推理
    outputs = model.chat(
        tokenizer=tokenizer,
        pixel_values=pixel_values,
        question=question, 
        generation_config=generation_config,
        num_patches_list=num_patches_list,
        IMG_START_TOKEN='<img>',
        IMG_END_TOKEN='</img>',
        IMG_CONTEXT_TOKEN='<IMG_CONTEXT>',
        verbose=False)
            
    print("="*20 + "output" + "="*20)
    print(outputs)
    y_pred.append(outputs[1].strip())
    print("="*20 + "real answers" + "="*20)
    print(answer)
    y_real.append(answer[0][1].strip())

Epoch 1/1:   0%|          | 1/3986 [00:01<1:38:51,  1.49s/it]

====================output====================
(A) Ate the medicine.
====================real answers====================
['(A) Ate the medicine.']


Epoch 1/1:   0%|          | 2/3986 [00:01<58:35,  1.13it/s]  

====================output====================
(C) Opened the closet/cabinet.
====================real answers====================
['(C) Opened the closet/cabinet.']


Epoch 1/1:   0%|          | 3/3986 [00:02<46:53,  1.42it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (12484 > 8192). Running this sequence through the model will result in indexing errors


...

Epoch 1/1: 100%|█████████▉| 3985/3986 [1:17:30<00:00,  1.13it/s]

====================output====================
(B) The rubber cube and the purple object collide
====================real answers====================
['(B) The rubber cube and the purple object collide']


Epoch 1/1: 100%|██████████| 3986/3986 [1:17:31<00:00,  1.13it/s]

====================output====================
(D) The gray cylinder collides with the brown object
====================real answers====================
['(C) The gray object collides with the sphere']


Epoch 1/1: 100%|██████████| 3986/3986 [1:17:31<00:00,  1.17s/it]


In [10]:
y_preds = [i.split('.')[0] for i in y_pred]

result = [1 if i == j else 0 for i, j in zip(y_real, y_preds)]

print("InternVL-2B-8-frames-MVBench, accuracy is: " + str(sum(result) / len(result)))

InternVL-2B-8-frames-MVBench, accuracy is: 0.5617160060210737
